# Risk Agent 風險評估測試

本測試展示：**風險管理 Agent 的完整工作流程**

使用 5 個自訂風險評估工具（@function_tool）進行手動測試：
- calculate_position_risk
- analyze_portfolio_concentration
- calculate_portfolio_risk
- perform_stress_test
- generate_risk_recommendations

## Step 1: 環境設置

In [1]:
import sys
from pathlib import Path
from dotenv import load_dotenv

from agents import (
    Runner,
    gen_trace_id,
    trace,
    WebSearchTool,
    CodeInterpreterTool,
)
from agents.mcp import MCPServerStdio

load_dotenv()

project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root / 'backend' / 'src'))

from common.logger import logger

print('✓ 環境設置完成')

2025-10-19 18:16:52 | INFO     | common.logger:setup_logger:100 | Logger initialized with level: INFO
2025-10-19 18:16:52 | INFO     | common.logger:setup_logger:102 | Logging to file: /Users/sacahan/Documents/workspace/CasualTrader/backend/logs/casualtrader.log


✓ 環境設置完成


## Step 2: 導入必要的模組

In [2]:
from trading.tools.risk_agent import (
    get_risk_agent,
)

print('✓ 模組導入完成')

✓ 模組導入完成


## Step 3: 初始化 OpenAI 內建工具

In [3]:
# WebSearchTool: 提供網路搜尋功能
web_search_tool = WebSearchTool(
    user_location=None,  # 可選：用戶位置，用於本地化搜尋結果
    filters=None,  # 可選：搜尋過濾器
    search_context_size="medium",  # 搜尋上下文大小：'low'、'medium'、'high'
)

# CodeInterpreterTool: 提供程式碼執行功能
# 必須指定 type 和 container 設置，container.type 必須為 "auto"
code_interpreter_tool = CodeInterpreterTool(
    tool_config={
        "type": "code_interpreter",
        "container": {
            "type": "auto"  # OpenAI 自動選擇最適合的容器
        },
    }
)

openai_tools = [web_search_tool, code_interpreter_tool]

## Step 4: 初始化 MCP Servers

In [4]:
# 全局變量來管理 MCP servers
casual_market_mcp = None
mcp_memory = None
mcp_servers = []

async def setup_mcp_servers():
    """設置 MCP servers"""
    global casual_market_mcp, mcp_memory, mcp_servers
    
    # 清理舊的連接
    if casual_market_mcp:
        try:
            await casual_market_mcp.__aexit__(None, None, None)
        except:
            pass
    
    if mcp_memory:
        try:
            await mcp_memory.__aexit__(None, None, None)
        except:
            pass
    
    # 初始化 casual-market-mcp
    casual_market_mcp = MCPServerStdio(
        params={
            "command": "uvx",
            "args": [
                "--from",
                "/Users/sacahan/Documents/workspace/CasualMarket",
                "casual-market-mcp",
            ],
        }, 
        name="casual_market_mcp", 
        client_session_timeout_seconds=120  # 增加超時時間
    )
    
    # 初始化 mcp-memory-libsql
    mcp_memory = MCPServerStdio(
        params={
            "command": "npx",
            "args": ["-y", "mcp-memory-libsql"],
            "env": {"LIBSQL_URL": "file:./labs/risk_agent.db"},
        },
        name="mcp_memory",
        client_session_timeout_seconds=120,  # 增加超時時間
    )
    
    # 連接 servers
    await casual_market_mcp.__aenter__()
    await mcp_memory.__aenter__()
    
    mcp_servers = [casual_market_mcp, mcp_memory]
    print('✓ MCP Servers 已連接')

await setup_mcp_servers()

✓ MCP Servers 已連接


## Step 5: 初始化 Risk Agent

In [5]:
# 創建風險管理 Agent
risk_agent = await get_risk_agent(
    model_name="gpt-4o-mini",
    mcp_servers=mcp_servers,
    openai_tools=openai_tools,
    max_turns=15,
)

print('✓ Risk Agent 已初始化')

2025-10-19 18:17:06 | INFO     | trading.tools.risk_agent:get_risk_agent:552 | get_risk_agent() called with model=gpt-4o-mini
2025-10-19 18:17:06 | INFO     | trading.tools.risk_agent:get_risk_agent:567 | Creating Agent with model=gpt-4o-mini, mcp_servers=2, tools=7
2025-10-19 18:17:06 | INFO     | trading.tools.risk_agent:get_risk_agent:580 | Risk Manager Agent created successfully


✓ Risk Agent 已初始化


## 測試 1: 單一投資組合風險評估

In [6]:
async def test_single_portfolio_risk():
    print('\n' + '='*70)
    print('測試 1: 單一投資組合風險評估')
    print('='*70)
    
    prompt = '''請評估我的投資組合風險。
    
投資組合配置：
- 台積電 (2330)：500 萬，目前股價 450 元，平均成本 420 元
  波動率 25%，Beta 1.1

- 聯發科 (2454)：300 萬，目前股價 900 元，平均成本 850 元
  波動率 30%，Beta 1.3

- 廣達 (2382)：200 萬，目前股價 150 元，平均成本 140 元
  波動率 28%，Beta 1.2

總投資組合價值：1,000 萬元

請進行以下分析：
1. 計算每個部位的風險指標
2. 分析投資組合的集中度
3. 計算整體投資組合風險
4. 執行壓力測試
5. 給出風險管理建議'''
    
    print(f'\n用戶提示:\n{prompt}')
    print('\n→ Agent 執行分析...')
    print('-' * 70)
    
    trace_id = gen_trace_id()
    try:
        with trace(workflow_name="risk_agent_lab", trace_id=trace_id):
            result = await Runner.run(risk_agent, prompt, max_turns=15)
    
        print('-' * 70)
        print('\nAgent 分析結果:')
        print(result.final_output)
        
        return result
    except Exception as e:
        print(f"執行錯誤: {e}")
        print(f"錯誤類型: {type(e).__name__}")
        return None

result_1 = await test_single_portfolio_risk()


測試 1: 單一投資組合風險評估

用戶提示:
請評估我的投資組合風險。

投資組合配置：
- 台積電 (2330)：500 萬，目前股價 450 元，平均成本 420 元
  波動率 25%，Beta 1.1

- 聯發科 (2454)：300 萬，目前股價 900 元，平均成本 850 元
  波動率 30%，Beta 1.3

- 廣達 (2382)：200 萬，目前股價 150 元，平均成本 140 元
  波動率 28%，Beta 1.2

總投資組合價值：1,000 萬元

請進行以下分析：
1. 計算每個部位的風險指標
2. 分析投資組合的集中度
3. 計算整體投資組合風險
4. 執行壓力測試
5. 給出風險管理建議

→ Agent 執行分析...
----------------------------------------------------------------------


2025-10-19 18:17:13 | INFO     | httpx._client:_send_single_request:1740 | HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-10-19 18:17:14 | INFO     | trading.tools.risk_agent:calculate_position_risk:204 | 開始計算部位風險 | 股票: 2330
2025-10-19 18:17:14 | INFO     | trading.tools.risk_agent:calculate_position_risk:226 | 部位風險計算完成 | 股票: 2330 | 風險評分: 28.0 | VaR(95%): 2,062,479 | 波動率: 25.00%
2025-10-19 18:17:15 | INFO     | httpx._client:_send_single_request:1025 | HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
2025-10-19 18:17:18 | INFO     | httpx._client:_send_single_request:1740 | HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-10-19 18:17:18 | INFO     | trading.tools.risk_agent:calculate_position_risk:204 | 開始計算部位風險 | 股票: 2454
2025-10-19 18:17:18 | INFO     | trading.tools.risk_agent:calculate_position_risk:226 | 部位風險計算完成 | 股票: 2454 | 風險評分: 39.0 | VaR(95%): 1,484,852 | 波動率: 30.00%
2025-10-19 18:

----------------------------------------------------------------------

Agent 分析結果:
### 投資組合風險評估

#### 1. 個別部位的風險指標
- **台積電 (2330)**
  - 目前價值: 4,999,950 元
  - 未實現盈虧: 333,330 元 (7.14%)
  - VaR (95%): 2,062,479 元
  - 最大回撤: 2,499,975 元
  - 風險分數: 28

- **聯發科 (2454)**
  - 目前價值: 2,999,700 元
  - 未實現盈虧: 166,650 元 (5.88%)
  - VaR (95%): 1,484,851 元
  - 最大回撤: 1,799,820 元
  - 風險分數: 39

- **廣達 (2382)**
  - 目前價值: 199,950 元
  - 未實現盈虧: 13,330 元 (7.14%)
  - VaR (95%): 92,377 元
  - 最大回撤: 111,972 元
  - 風險分數: 34

---

#### 2. 投資組合集中度分析
- **HHI 指數**: 0.34 (高度集中)
- **有效股票數**: 2.94
- **前五名集中度**: 81.996%
- **最大部位權重**: 49.999%
- **行業集中度**:
  - 半導體: 79.996%
  - 科技: 1.9995%
- **風險評估**: 集中度風險偏高，建議分散。

---

#### 3. 整體投資組合風險
- **總價值**: 10,000,000 元
- **投資組合波動率**: 22.06%
- **Beta**: 0.96
- **投資組合 VaR (95%)**: 3,639,708 元
- **整體風險分數**: 39.8
- **風險等級**: 中低風險

---

#### 4. 壓力測試結果
- **市場回調 10%**
  - 損失: 819,960 元 (10%)
  - 嚴重程度: 嚴重

- **市場回調 20%**
  - 損失: 1,639,920 元 (20%)
  - 嚴重程度: 極端
  
- **最大潛在損失**: 1,639,920 元

---

2025-10-19 18:17:59 | INFO     | httpx._client:_send_single_request:1025 | HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"


## 測試 2: 完整 Agent 工作流程 - 高風險投資組合

In [ ]:
async def test_high_risk_portfolio():
    print('\n' + '='*70)
    print('測試 2: 完整 Agent 工作流程 - 高風險投資組合')
    print('='*70)
    
    prompt = '''請評估這個高風險投資組合的風險狀況。
    
投資組合配置：
- 個股部位 1：600 萬，波動率 45%，Beta 1.8
- 個股部位 2：500 萬，波動率 50%，Beta 2.0
- 個股部位 3：400 萬，波動率 40%，Beta 1.7
- 個股部位 4：300 萬，波動率 48%，Beta 1.9
- 個股部位 5：200 萬，波動率 55%，Beta 2.2

總投資組合價值：2,000 萬元

市場條件：
- 整體市場波動率：30%
- 最近 1 個月市場下跌 8%
- 投資人情緒偏弱

請分析：
1. 投資組合是否過度集中？
2. 風險是否過高？
3. 在市場崩盤情況下的潛在損失
4. 立即需要採取的風險管理措施
5. 建議的避險策略'''
    
    print(f'\n用戶提示:\n{prompt}')
    print('\n→ Agent 執行分析...')
    print('-' * 70)
    
    trace_id = gen_trace_id()
    try:
        with trace(workflow_name="risk_agent_lab", trace_id=trace_id):
            result = await Runner.run(risk_agent, prompt, max_turns=15)
    
        print('-' * 70)
        print('\nAgent 分析結果:')
        print(result.final_output)
        
        return result
    except Exception as e:
        print(f"執行錯誤: {e}")
        print(f"錯誤類型: {type(e).__name__}")
        return None

result_2 = await test_high_risk_portfolio()

## 測試 3: 完整 Agent 工作流程 - 風險優化建議

In [ ]:
async def test_risk_optimization():
    print('\n' + '='*70)
    print('測試 3: 完整 Agent 工作流程 - 風險優化建議')
    print('='*70)
    
    prompt = '''我目前的投資組合存在風險集中問題，請提供優化建議。
    
當前投資組合（5,000 萬元）：
- 電子產業：3,000 萬（60%）
  * 台積電：1,500 萬
  * 聯發科：1,000 萬
  * 其他電子：500 萬

- 金融產業：1,500 萬（30%）
  * 國泰金：800 萬
  * 富邦金：700 萬

- 其他產業：500 萬（10%）
  * 大型鐵路：500 萬

風險特性：
- 主要部位與市場高度相關（Beta > 1.5）
- 產業集中度高
- 缺乏防守性資產

目標：
- 維持 8-12% 預期年報酬
- 將投資組合波動率從 32% 降至 22%
- 增加抗跌性

請提供：
1. 風險評估總結
2. 產業配置優化建議
3. 具體的調整方案
4. 預期的風險改善
5. 執行建議和時間表'''
    
    print(f'\n用戶提示:\n{prompt}')
    print('\n→ Agent 執行分析...')
    print('-' * 70)
    
    trace_id = gen_trace_id()
    try:
        with trace(workflow_name="risk_agent_lab", trace_id=trace_id):
            result = await Runner.run(risk_agent, prompt, max_turns=15)
    
        print('-' * 70)
        print('\nAgent 分析結果:')
        print(result.final_output)
        
        return result
    except Exception as e:
        print(f"執行錯誤: {e}")
        print(f"錯誤類型: {type(e).__name__}")
        return None
    
    return result

result_3 = await test_risk_optimization()

## 清理資源

In [ ]:
async def cleanup_mcp_servers():
    """清理 MCP servers 連接"""
    global casual_market_mcp, mcp_memory
    
    try:
        if casual_market_mcp:
            await casual_market_mcp.__aexit__(None, None, None)
            print('✓ casual_market_mcp 已關閉')
        
        if mcp_memory:
            await mcp_memory.__aexit__(None, None, None)
            print('✓ mcp_memory 已關閉')
            
        print('✓ 所有 MCP servers 已清理完成')
        
    except Exception as e:
        print(f'清理過程中發生錯誤: {e}')

# 測試完成後執行清理
await cleanup_mcp_servers()